# 🧪 TRM Dataset Comparison - Faithful Reproduction (Fixed)

**完全忠实于 TinyRecursiveModels 原始实现**，仅修改：
- 优化器: AdamAtan2 → AdamW
- 单 GPU 运行（移除分布式代码）

**新增效率指标**:
- GPU 内存追踪
- FLOPs 估算
- EMA 评估支持

| 配置 | 谜题数 | 增强 | Epochs | Samples Seen |
|------|-------|-----|--------|-------------|
| `1k-aug1000` | 1K | 1000× | **2** | ~2.0M |
| `10k-aug100` | 10K | 100× | **2** | ~2.0M |
| `100k-aug10` | 100K | 10× | **2** | ~2.2M |
| `1M-aug0` | 1M | 0× | **2** | ~2.0M |

**注意**: 所有配置设置为2 epochs，确保看到相同的~2M样本，实现公平对比。

**v2 修复内容**:
- 修复 EMA + torch.compile 兼容性问题
- 添加 CUDA 缓存清理
- 修复评估循环中的内存泄漏
- 添加更好的错误处理

---
## 1️⃣ Configuration

In [ ]:
!pip install -q torch einops tqdm numpy pydantic wandb coolname huggingface_hub
print("✅ Dependencies installed")

In [ ]:
# ============================================================================
# 🔑 API Tokens
# ============================================================================
WANDB_API_KEY = ''
HF_TOKEN = ""

import os
os.environ["HF_TOKEN"] = HF_TOKEN

import wandb
wandb.login(key=WANDB_API_KEY)
print("✅ Logged in")

In [ ]:
# ============================================================================
# 🎯 EXPERIMENT CONFIGURATION
# ============================================================================
SELECTED_CONFIG = "1k-aug1000"  # "1k-aug1000", "10k-aug100", "100k-aug10", "1M-aug0"
BATCH_SIZE = 1024
PROJECT_NAME = "TRM-DataVS"
FORCE_REBUILD = False
USE_EMA = True  # 使用 EMA 评估（原始实现支持）
MAX_EVAL_BATCHES = 100  # 限制评估batch数量，节省时间（设置为None评估完整数据集）

DATASET_CONFIGS = {
    # 所有配置设置为2 epochs，确保看到相同的~2M样本（公平对比）
    # 计算公式: epochs = 2,000,000 / (total_groups × mean_puzzle_examples)
    "1k-aug1000": {"subsample": 1000, "augmentation": 1000, "epochs": 20, "desc": "1K × 1000 aug (baseline)"},
    "10k-aug100": {"subsample": 10000, "augmentation": 100, "epochs": 20, "desc": "10K × 100 aug"},
    "100k-aug10": {"subsample": 100000, "augmentation": 10, "epochs": 20, "desc": "100K × 10 aug"},
    "1M-aug0": {"subsample": 1000000, "augmentation": 0, "epochs": 20, "desc": "1M, no aug"},
}

cfg = DATASET_CONFIGS[SELECTED_CONFIG]
print(f"📊 Config: {SELECTED_CONFIG} - {cfg['desc']}")
print(f"   Epochs: {cfg['epochs']}, Batch: {BATCH_SIZE}, EMA: {USE_EMA}")
print(f"   Max Eval Batches: {MAX_EVAL_BATCHES if MAX_EVAL_BATCHES else 'Full dataset'}")

---
## 2️⃣ Core Implementation (Faithful to TinyRecursiveModels)

In [ ]:
# ============================================================================
# Imports & GPU Setup
# ============================================================================
import os, sys, math, json, csv, copy, time, shutil
from typing import Optional, Any, Sequence, List, Dict, Tuple, Union
from dataclasses import dataclass

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, IterableDataset, get_worker_info
from torch.optim.optimizer import Optimizer, ParamsT
from tqdm import tqdm
import pydantic
from pydantic import BaseModel
import einops
from torch.nn.functional import scaled_dot_product_attention

# GPU Setup (H100/A100 TF32)
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🖥️ GPU: {gpu_name}")
    if "H100" in gpu_name or "A100" in gpu_name:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print("🚀 TF32 enabled")

IGNORE_LABEL_ID = -100
print("✅ Imports complete")

In [ ]:
# ============================================================================
# Common Utils (models/common.py)
# ============================================================================
def trunc_normal_init_(tensor: torch.Tensor, std: float = 1.0, lower: float = -2.0, upper: float = 2.0):
    """Truncated normal init - JAX compatible (exact copy from TinyRecursiveModels)"""
    with torch.no_grad():
        if std == 0:
            tensor.zero_()
        else:
            sqrt2 = math.sqrt(2)
            a = math.erf(lower / sqrt2)
            b = math.erf(upper / sqrt2)
            z = (b - a) / 2
            c = (2 * math.pi) ** -0.5
            pdf_u = c * math.exp(-0.5 * lower ** 2)
            pdf_l = c * math.exp(-0.5 * upper ** 2)
            comp_std = std / math.sqrt(1 - (upper * pdf_u - lower * pdf_l) / z - ((pdf_u - pdf_l) / z) ** 2)
            tensor.uniform_(a, b)
            tensor.erfinv_()
            tensor.mul_(sqrt2 * comp_std)
            tensor.clip_(lower * comp_std, upper * comp_std)
    return tensor

print("✅ Common utils loaded")

In [ ]:
# ============================================================================
# Layers (models/layers.py) - Exact copy
# ============================================================================
CosSin = Tuple[torch.Tensor, torch.Tensor]

def _find_multiple(a, b):
    return (-(a // -b)) * b

def rotate_half(x: torch.Tensor):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(q: torch.Tensor, k: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor):
    orig_dtype = q.dtype
    q = q.to(cos.dtype)
    k = k.to(cos.dtype)
    q_embed = (q * cos.unsqueeze(-2)) + (rotate_half(q) * sin.unsqueeze(-2))
    k_embed = (k * cos.unsqueeze(-2)) + (rotate_half(k) * sin.unsqueeze(-2))
    return q_embed.to(orig_dtype), k_embed.to(orig_dtype)

def rms_norm(hidden_states: torch.Tensor, variance_epsilon: float) -> torch.Tensor:
    input_dtype = hidden_states.dtype
    hidden_states = hidden_states.to(torch.float32)
    variance = hidden_states.square().mean(-1, keepdim=True)
    hidden_states = hidden_states * torch.rsqrt(variance + variance_epsilon)
    return hidden_states.to(input_dtype)

class CastedLinear(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool):
        super().__init__()
        self.weight = nn.Parameter(trunc_normal_init_(torch.empty((out_features, in_features)), std=1.0 / (in_features ** 0.5)))
        self.bias = nn.Parameter(torch.zeros((out_features,))) if bias else None
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return F.linear(input, self.weight.to(input.dtype), bias=self.bias.to(input.dtype) if self.bias is not None else None)

class CastedEmbedding(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int, init_std: float, cast_to: torch.dtype):
        super().__init__()
        self.cast_to = cast_to
        self.embedding_weight = nn.Parameter(trunc_normal_init_(torch.empty((num_embeddings, embedding_dim)), std=init_std))
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return F.embedding(input, self.embedding_weight.to(self.cast_to))

class RotaryEmbedding(nn.Module):
    def __init__(self, dim, max_position_embeddings, base, device=None):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2, dtype=torch.float32, device=device) / dim))
        t = torch.arange(max_position_embeddings, dtype=torch.float32, device=device)
        freqs = torch.outer(t, inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        self.cos_cached = nn.Buffer(emb.cos(), persistent=False)
        self.sin_cached = nn.Buffer(emb.sin(), persistent=False)
    def forward(self):
        return self.cos_cached, self.sin_cached

class Attention(nn.Module):
    def __init__(self, hidden_size, head_dim, num_heads, num_key_value_heads, causal=False):
        super().__init__()
        self.hidden_size = hidden_size
        self.head_dim = head_dim
        self.output_size = head_dim * num_heads
        self.num_heads = num_heads
        self.num_key_value_heads = num_key_value_heads
        self.causal = causal
        self.qkv_proj = CastedLinear(self.hidden_size, (self.num_heads + 2 * self.num_key_value_heads) * self.head_dim, bias=False)
        self.o_proj = CastedLinear(self.output_size, self.hidden_size, bias=False)

    def forward(self, cos_sin: CosSin, hidden_states: torch.Tensor) -> torch.Tensor:
        batch_size, seq_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        qkv = qkv.view(batch_size, seq_len, self.num_heads + 2 * self.num_key_value_heads, self.head_dim)
        query = qkv[:, :, :self.num_heads]
        key = qkv[:, :, self.num_heads: self.num_heads + self.num_key_value_heads]
        value = qkv[:, :, self.num_heads + self.num_key_value_heads:]
        if cos_sin is not None:
            cos, sin = cos_sin
            query, key = apply_rotary_pos_emb(query, key, cos, sin)
        query, key, value = map(lambda t: einops.rearrange(t, 'B S H D -> B H S D'), (query, key, value))
        attn_output = scaled_dot_product_attention(query=query, key=key, value=value, is_causal=self.causal)
        attn_output = einops.rearrange(attn_output, 'B H S D -> B S H D')
        attn_output = attn_output.view(batch_size, seq_len, self.output_size)
        return self.o_proj(attn_output)

class SwiGLU(nn.Module):
    def __init__(self, hidden_size: int, expansion: float):
        super().__init__()
        inter = _find_multiple(round(expansion * hidden_size * 2 / 3), 256)
        self.gate_up_proj = CastedLinear(hidden_size, inter * 2, bias=False)
        self.down_proj = CastedLinear(inter, hidden_size, bias=False)
    def forward(self, x):
        gate, up = self.gate_up_proj(x).chunk(2, dim=-1)
        return self.down_proj(F.silu(gate) * up)

print("✅ Layers loaded")

In [ ]:
# ============================================================================
# Sparse Embedding (models/sparse_embedding.py) - Exact copy
# ============================================================================
class CastedSparseEmbedding(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int, batch_size: int, init_std: float, cast_to: torch.dtype):
        super().__init__()
        self.cast_to = cast_to
        self.weights = nn.Buffer(trunc_normal_init_(torch.empty((num_embeddings, embedding_dim)), std=init_std), persistent=True)
        self.local_weights = nn.Buffer(torch.zeros(batch_size, embedding_dim, requires_grad=True), persistent=False)
        self.local_ids = nn.Buffer(torch.zeros(batch_size, dtype=torch.int32), persistent=False)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        if not self.training:
            return self.weights[inputs].to(self.cast_to)
        with torch.no_grad():
            self.local_weights.copy_(self.weights[inputs])
            self.local_ids.copy_(inputs)
        return self.local_weights.to(self.cast_to)

class CastedSparseEmbeddingSignSGD(Optimizer):
    """Single GPU version of CastedSparseEmbeddingSignSGD_Distributed"""
    def __init__(self, params: ParamsT, lr: Union[float, torch.Tensor] = 1e-3, weight_decay: float = 1e-2):
        defaults = dict(lr=lr, weight_decay=weight_decay)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        for group in self.param_groups:
            local_weights_grad, local_ids, weights = None, None, None
            assert len(group["params"]) == 3
            for p in group["params"]:
                if p.requires_grad: local_weights_grad = p.grad
                elif p.ndim == 1: local_ids = p
                elif p.ndim == 2: weights = p
            if local_weights_grad is None or local_ids is None or weights is None: continue
            # SignSGD update
            N, D = local_weights_grad.shape
            grad_ids, inv = local_ids.unique(return_inverse=True)
            grad = torch.zeros((grad_ids.shape[0], D), dtype=local_weights_grad.dtype, device=local_weights_grad.device)
            grad.scatter_add_(0, inv.unsqueeze(-1).expand(-1, D), local_weights_grad)
            p = weights[grad_ids]
            p.mul_(1.0 - group["lr"] * group["weight_decay"]).add_(torch.sign(grad), alpha=-group["lr"])
            weights[grad_ids] = p

print("✅ Sparse embedding loaded")

In [ ]:
# ============================================================================
# EMA Helper (models/ema.py) - Fixed for torch.compile compatibility
# ============================================================================
class EMAHelper(object):
    def __init__(self, mu=0.999):
        self.mu = mu
        self.shadow = {}

    def _unwrap_module(self, module):
        """Unwrap DataParallel or compiled module"""
        if isinstance(module, nn.DataParallel):
            module = module.module
        # Handle torch.compile wrapped modules
        if hasattr(module, '_orig_mod'):
            return module._orig_mod
        return module

    def register(self, module):
        module = self._unwrap_module(module)
        for name, param in module.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self, module):
        module = self._unwrap_module(module)
        for name, param in module.named_parameters():
            if param.requires_grad:
                self.shadow[name].data = (1. - self.mu) * param.data + self.mu * self.shadow[name].data

    def ema(self, module):
        module = self._unwrap_module(module)
        for name, param in module.named_parameters():
            if param.requires_grad:
                param.data.copy_(self.shadow[name].data)

    def ema_copy(self, module):
        """Create a copy with EMA weights - handles compiled models"""
        # For compiled models, we need special handling
        try:
            module_copy = copy.deepcopy(module)
        except Exception as e:
            # If deepcopy fails (e.g., with compiled models), try alternative approach
            print(f"⚠️ deepcopy failed, using state_dict approach: {e}")
            # Get the underlying module
            orig_module = self._unwrap_module(module)
            module_copy = copy.deepcopy(orig_module)
            # Re-compile if the original was compiled
            if hasattr(module, '_orig_mod') and "DISABLE_COMPILE" not in os.environ:
                module_copy = torch.compile(module_copy)
        
        self.ema(module_copy)
        return module_copy

    def state_dict(self): return self.shadow
    def load_state_dict(self, state_dict): self.shadow = state_dict

print("✅ EMA helper loaded")

In [ ]:
# ============================================================================
# TRM Model (models/recursive_reasoning/trm.py) - Exact copy
# ============================================================================
@dataclass
class TinyRecursiveReasoningModel_ACTV1InnerCarry:
    z_H: torch.Tensor
    z_L: torch.Tensor

@dataclass
class TinyRecursiveReasoningModel_ACTV1Carry:
    inner_carry: TinyRecursiveReasoningModel_ACTV1InnerCarry
    steps: torch.Tensor
    halted: torch.Tensor
    current_data: Dict[str, torch.Tensor]

class TinyRecursiveReasoningModel_ACTV1Config(BaseModel):
    batch_size: int; seq_len: int; puzzle_emb_ndim: int = 0; num_puzzle_identifiers: int; vocab_size: int
    H_cycles: int; L_cycles: int; H_layers: int; L_layers: int
    hidden_size: int; expansion: float; num_heads: int; pos_encodings: str
    rms_norm_eps: float = 1e-5; rope_theta: float = 10000.0
    halt_max_steps: int; halt_exploration_prob: float; forward_dtype: str = "bfloat16"
    mlp_t: bool = False; puzzle_emb_len: int = 16; no_ACT_continue: bool = True

class TinyRecursiveReasoningModel_ACTV1Block(nn.Module):
    def __init__(self, config: TinyRecursiveReasoningModel_ACTV1Config):
        super().__init__()
        self.config = config
        if self.config.mlp_t:
            self.puzzle_emb_len = -(self.config.puzzle_emb_ndim // -self.config.hidden_size) if self.config.puzzle_emb_len == 0 else self.config.puzzle_emb_len
            self.mlp_t = SwiGLU(hidden_size=self.config.seq_len + self.puzzle_emb_len, expansion=config.expansion)
        else:
            self.self_attn = Attention(hidden_size=config.hidden_size, head_dim=config.hidden_size // config.num_heads,
                                       num_heads=config.num_heads, num_key_value_heads=config.num_heads, causal=False)
        self.mlp = SwiGLU(hidden_size=config.hidden_size, expansion=config.expansion)
        self.norm_eps = config.rms_norm_eps

    def forward(self, cos_sin: CosSin, hidden_states: torch.Tensor) -> torch.Tensor:
        if self.config.mlp_t:
            hidden_states = hidden_states.transpose(1, 2)
            out = self.mlp_t(hidden_states)
            hidden_states = rms_norm(hidden_states + out, variance_epsilon=self.norm_eps)
            hidden_states = hidden_states.transpose(1, 2)
        else:
            hidden_states = rms_norm(hidden_states + self.self_attn(cos_sin=cos_sin, hidden_states=hidden_states), variance_epsilon=self.norm_eps)
        out = self.mlp(hidden_states)
        hidden_states = rms_norm(hidden_states + out, variance_epsilon=self.norm_eps)
        return hidden_states

class TinyRecursiveReasoningModel_ACTV1ReasoningModule(nn.Module):
    def __init__(self, layers: List[TinyRecursiveReasoningModel_ACTV1Block]):
        super().__init__()
        self.layers = nn.ModuleList(layers)
    def forward(self, hidden_states: torch.Tensor, input_injection: torch.Tensor, **kwargs) -> torch.Tensor:
        hidden_states = hidden_states + input_injection
        for layer in self.layers:
            hidden_states = layer(hidden_states=hidden_states, **kwargs)
        return hidden_states

class TinyRecursiveReasoningModel_ACTV1_Inner(nn.Module):
    def __init__(self, config: TinyRecursiveReasoningModel_ACTV1Config):
        super().__init__()
        self.config = config
        self.forward_dtype = getattr(torch, self.config.forward_dtype)
        self.embed_scale = math.sqrt(self.config.hidden_size)
        embed_init_std = 1.0 / self.embed_scale
        self.embed_tokens = CastedEmbedding(self.config.vocab_size, self.config.hidden_size, init_std=embed_init_std, cast_to=self.forward_dtype)
        self.lm_head = CastedLinear(self.config.hidden_size, self.config.vocab_size, bias=False)
        self.q_head = CastedLinear(self.config.hidden_size, 2, bias=True)
        self.puzzle_emb_len = -(self.config.puzzle_emb_ndim // -self.config.hidden_size) if self.config.puzzle_emb_len == 0 else self.config.puzzle_emb_len
        if self.config.puzzle_emb_ndim > 0:
            self.puzzle_emb = CastedSparseEmbedding(self.config.num_puzzle_identifiers, self.config.puzzle_emb_ndim,
                                                    batch_size=self.config.batch_size, init_std=0, cast_to=self.forward_dtype)
        if self.config.pos_encodings == "rope":
            self.rotary_emb = RotaryEmbedding(dim=self.config.hidden_size // self.config.num_heads,
                                              max_position_embeddings=self.config.seq_len + self.puzzle_emb_len, base=self.config.rope_theta)
        elif self.config.pos_encodings == "learned":
            self.embed_pos = CastedEmbedding(self.config.seq_len + self.puzzle_emb_len, self.config.hidden_size, init_std=embed_init_std, cast_to=self.forward_dtype)
        self.L_level = TinyRecursiveReasoningModel_ACTV1ReasoningModule(
            layers=[TinyRecursiveReasoningModel_ACTV1Block(self.config) for _ in range(self.config.L_layers)])
        self.H_init = nn.Buffer(trunc_normal_init_(torch.empty(self.config.hidden_size, dtype=self.forward_dtype), std=1), persistent=True)
        self.L_init = nn.Buffer(trunc_normal_init_(torch.empty(self.config.hidden_size, dtype=self.forward_dtype), std=1), persistent=True)
        with torch.no_grad():
            self.q_head.weight.zero_()
            self.q_head.bias.fill_(-5)

    def _input_embeddings(self, input: torch.Tensor, puzzle_identifiers: torch.Tensor):
        embedding = self.embed_tokens(input.to(torch.int32))
        if self.config.puzzle_emb_ndim > 0:
            puzzle_embedding = self.puzzle_emb(puzzle_identifiers)
            pad_count = self.puzzle_emb_len * self.config.hidden_size - puzzle_embedding.shape[-1]
            if pad_count > 0:
                puzzle_embedding = F.pad(puzzle_embedding, (0, pad_count))
            embedding = torch.cat((puzzle_embedding.view(-1, self.puzzle_emb_len, self.config.hidden_size), embedding), dim=-2)
        if self.config.pos_encodings == "learned":
            embedding = 0.707106781 * (embedding + self.embed_pos.embedding_weight.to(self.forward_dtype))
        return self.embed_scale * embedding

    def empty_carry(self, batch_size: int):
        return TinyRecursiveReasoningModel_ACTV1InnerCarry(
            z_H=torch.empty(batch_size, self.config.seq_len + self.puzzle_emb_len, self.config.hidden_size, dtype=self.forward_dtype),
            z_L=torch.empty(batch_size, self.config.seq_len + self.puzzle_emb_len, self.config.hidden_size, dtype=self.forward_dtype))

    def reset_carry(self, reset_flag: torch.Tensor, carry: TinyRecursiveReasoningModel_ACTV1InnerCarry):
        return TinyRecursiveReasoningModel_ACTV1InnerCarry(
            z_H=torch.where(reset_flag.view(-1, 1, 1), self.H_init, carry.z_H),
            z_L=torch.where(reset_flag.view(-1, 1, 1), self.L_init, carry.z_L))

    def forward(self, carry: TinyRecursiveReasoningModel_ACTV1InnerCarry, batch: Dict[str, torch.Tensor]):
        seq_info = dict(cos_sin=self.rotary_emb() if hasattr(self, "rotary_emb") else None)
        input_embeddings = self._input_embeddings(batch["inputs"], batch["puzzle_identifiers"])
        z_H, z_L = carry.z_H, carry.z_L
        with torch.no_grad():
            for _H_step in range(self.config.H_cycles - 1):
                for _L_step in range(self.config.L_cycles):
                    z_L = self.L_level(z_L, z_H + input_embeddings, **seq_info)
                z_H = self.L_level(z_H, z_L, **seq_info)
        for _L_step in range(self.config.L_cycles):
            z_L = self.L_level(z_L, z_H + input_embeddings, **seq_info)
        z_H = self.L_level(z_H, z_L, **seq_info)
        new_carry = TinyRecursiveReasoningModel_ACTV1InnerCarry(z_H=z_H.detach(), z_L=z_L.detach())
        output = self.lm_head(z_H)[:, self.puzzle_emb_len:]
        q_logits = self.q_head(z_H[:, 0]).to(torch.float32)
        return new_carry, output, (q_logits[..., 0], q_logits[..., 1])

class TinyRecursiveReasoningModel_ACTV1(nn.Module):
    def __init__(self, config_dict: dict):
        super().__init__()
        self.config = TinyRecursiveReasoningModel_ACTV1Config(**config_dict)
        self.inner = TinyRecursiveReasoningModel_ACTV1_Inner(self.config)

    @property
    def puzzle_emb(self): return self.inner.puzzle_emb

    def initial_carry(self, batch: Dict[str, torch.Tensor]):
        batch_size = batch["inputs"].shape[0]
        return TinyRecursiveReasoningModel_ACTV1Carry(
            inner_carry=self.inner.empty_carry(batch_size),
            steps=torch.zeros((batch_size,), dtype=torch.int32),
            halted=torch.ones((batch_size,), dtype=torch.bool),
            current_data={k: torch.empty_like(v) for k, v in batch.items()})

    def forward(self, carry: TinyRecursiveReasoningModel_ACTV1Carry, batch: Dict[str, torch.Tensor]):
        new_inner_carry = self.inner.reset_carry(carry.halted, carry.inner_carry)
        new_steps = torch.where(carry.halted, 0, carry.steps)
        new_current_data = {k: torch.where(carry.halted.view((-1,) + (1,) * (batch[k].ndim - 1)), batch[k], v) for k, v in carry.current_data.items()}
        new_inner_carry, logits, (q_halt_logits, q_continue_logits) = self.inner(new_inner_carry, new_current_data)
        outputs = {"logits": logits, "q_halt_logits": q_halt_logits, "q_continue_logits": q_continue_logits}
        with torch.no_grad():
            new_steps = new_steps + 1
            is_last_step = new_steps >= self.config.halt_max_steps
            halted = is_last_step
            if self.training and (self.config.halt_max_steps > 1):
                if self.config.no_ACT_continue:
                    halted = halted | (q_halt_logits > 0)
                else:
                    halted = halted | (q_halt_logits > q_continue_logits)
                min_halt_steps = (torch.rand_like(q_halt_logits) < self.config.halt_exploration_prob) * torch.randint_like(new_steps, low=2, high=self.config.halt_max_steps + 1)
                halted = halted & (new_steps >= min_halt_steps)
                if not self.config.no_ACT_continue:
                    _, _, (next_q_halt, next_q_cont) = self.inner(new_inner_carry, new_current_data)
                    outputs["target_q_continue"] = torch.sigmoid(torch.where(is_last_step, next_q_halt, torch.maximum(next_q_halt, next_q_cont)))
        return TinyRecursiveReasoningModel_ACTV1Carry(new_inner_carry, new_steps, halted, new_current_data), outputs

print("✅ TRM Model loaded")

In [ ]:
# ============================================================================
# Loss Functions (models/losses.py) - Exact copy
# ============================================================================
def s(x, epsilon=1e-30):
    return torch.where(x < 0, 1 / (1 - x + epsilon), x + 1)

def log_stablemax(x, dim=-1):
    s_x = s(x)
    return torch.log(s_x / torch.sum(s_x, dim=dim, keepdim=True))

def stablemax_cross_entropy(logits, labels, ignore_index: int = -100, valid_mask=None):
    logprobs = log_stablemax(logits.to(torch.float64), dim=-1)
    if valid_mask is None:
        valid_mask = (labels != ignore_index)
    transformed_labels = torch.where(valid_mask, labels, 0)
    prediction_logprobs = torch.gather(logprobs, index=transformed_labels.to(torch.long).unsqueeze(-1), dim=-1).squeeze(-1)
    return -torch.where(valid_mask, prediction_logprobs, 0)

class ACTLossHead(nn.Module):
    def __init__(self, model: nn.Module, loss_type: str):
        super().__init__()
        self.model = model
        self.loss_fn = stablemax_cross_entropy  # globals()[loss_type]

    def initial_carry(self, *args, **kwargs):
        return self.model.initial_carry(*args, **kwargs)

    def forward(self, return_keys: Sequence[str], **model_kwargs):
        new_carry, outputs = self.model(**model_kwargs)
        labels = new_carry.current_data["labels"]
        with torch.no_grad():
            outputs["preds"] = torch.argmax(outputs["logits"], dim=-1)
            mask = (labels != IGNORE_LABEL_ID)
            loss_counts = mask.sum(-1)
            loss_divisor = loss_counts.clamp_min(1).unsqueeze(-1)
            is_correct = mask & (torch.argmax(outputs["logits"], dim=-1) == labels)
            seq_is_correct = is_correct.sum(-1) == loss_counts
            valid_metrics = new_carry.halted & (loss_counts > 0)
            metrics = {
                "count": valid_metrics.sum(),
                "accuracy": torch.where(valid_metrics, (is_correct.to(torch.float32) / loss_divisor).sum(-1), 0).sum(),
                "exact_accuracy": (valid_metrics & seq_is_correct).sum(),
                "q_halt_accuracy": (valid_metrics & ((outputs["q_halt_logits"] >= 0) == seq_is_correct)).sum(),
                "steps": torch.where(valid_metrics, new_carry.steps, 0).sum(),
            }
        lm_loss = (self.loss_fn(outputs["logits"], labels, ignore_index=IGNORE_LABEL_ID, valid_mask=mask) / loss_divisor).sum()
        q_halt_loss = F.binary_cross_entropy_with_logits(outputs["q_halt_logits"], seq_is_correct.to(outputs["q_halt_logits"].dtype), reduction="sum")
        metrics.update({"lm_loss": lm_loss.detach(), "q_halt_loss": q_halt_loss.detach()})
        q_continue_loss = 0
        if "target_q_continue" in outputs:
            q_continue_loss = F.binary_cross_entropy_with_logits(outputs["q_continue_logits"], outputs["target_q_continue"], reduction="sum")
            metrics["q_continue_loss"] = q_continue_loss.detach()
        detached_outputs = {k: outputs[k].detach() for k in return_keys if k in outputs}
        return new_carry, lm_loss + 0.5 * (q_halt_loss + q_continue_loss), metrics, detached_outputs, new_carry.halted.all()

print("✅ Loss functions loaded")

In [ ]:
# ============================================================================
# Dataset (puzzle_dataset.py) - Exact copy
# ============================================================================
class PuzzleDatasetMetadata(pydantic.BaseModel):
    pad_id: int; ignore_label_id: Optional[int]; blank_identifier_id: int; vocab_size: int; seq_len: int
    num_puzzle_identifiers: int; total_groups: int; mean_puzzle_examples: float; total_puzzles: int; sets: List[str]

class PuzzleDatasetConfig(pydantic.BaseModel):
    seed: int; dataset_paths: List[str]; global_batch_size: int; test_set_mode: bool; epochs_per_iter: int
    rank: int = 0; num_replicas: int = 1; shuffle_eval: bool = False; eval_seed: Optional[int] = None

def _sample_batch(rng, group_order, puzzle_indices, group_indices, start_index, global_batch_size):
    batch, batch_puzzle_indices, current_size = [], [], 0
    while (start_index < group_order.size) and (current_size < global_batch_size):
        group_id = group_order[start_index]
        puzzle_id = rng.integers(group_indices[group_id], group_indices[group_id + 1])
        start_index += 1
        puzzle_start = puzzle_indices[puzzle_id]
        puzzle_size = int(puzzle_indices[puzzle_id + 1] - puzzle_start)
        append_size = min(puzzle_size, global_batch_size - current_size)
        batch_puzzle_indices.append(np.full(append_size, puzzle_id, dtype=np.int32))
        batch.append(puzzle_start + rng.choice(puzzle_size, append_size, replace=False))
        current_size += append_size
    return start_index, np.concatenate(batch), np.concatenate(batch_puzzle_indices)

class PuzzleDataset(IterableDataset):
    def __init__(self, config: PuzzleDatasetConfig, split: str = "train"):
        super().__init__()
        self.config, self.split = config, split
        self._load_metadata()
        self.local_batch_size = config.global_batch_size // config.num_replicas
        self._data, self._iters = None, 0

    def _load_metadata(self):
        prev_seq_len, mean_puzzle_examples, total_puzzles, total_groups = None, 0, 0, 0
        for dataset_path in self.config.dataset_paths:
            with open(os.path.join(dataset_path, self.split, "dataset.json"), "r") as f:
                current = PuzzleDatasetMetadata(**json.load(f))
            if prev_seq_len is None: self._base = current; prev_seq_len = current.seq_len
            mean_puzzle_examples += current.mean_puzzle_examples * current.total_puzzles
            total_puzzles += current.total_puzzles; total_groups += current.total_groups
        self.metadata = PuzzleDatasetMetadata(seq_len=self._base.seq_len, vocab_size=self._base.vocab_size, pad_id=self._base.pad_id,
            ignore_label_id=self._base.ignore_label_id, blank_identifier_id=self._base.blank_identifier_id,
            num_puzzle_identifiers=self._base.num_puzzle_identifiers, total_groups=total_groups,
            mean_puzzle_examples=mean_puzzle_examples / total_puzzles if total_puzzles else 0, total_puzzles=total_puzzles, sets=self._base.sets)

    def _lazy_load(self):
        if self._data: return
        self._data = {}
        for set_name in self.metadata.sets:
            for i, dp in enumerate(self.config.dataset_paths):
                sn = set_name + str(i) if i else set_name
                self._data[sn] = {fn: np.load(os.path.join(dp, self.split, f"{set_name}__{fn}.npy"), mmap_mode="r" if fn in ["inputs","labels"] else None)
                    for fn in ["inputs","labels","puzzle_identifiers","puzzle_indices","group_indices"]}

    def _collate(self, batch):
        batch = {k: v.astype(np.int32) for k, v in batch.items()}
        if self.metadata.ignore_label_id is not None:
            batch["labels"][batch["labels"] == self.metadata.ignore_label_id] = IGNORE_LABEL_ID
        if batch["puzzle_identifiers"].size < self.local_batch_size:
            pad_size = self.local_batch_size - batch["puzzle_identifiers"].size
            pad_values = {"inputs": self.metadata.pad_id, "labels": IGNORE_LABEL_ID, "puzzle_identifiers": self.metadata.blank_identifier_id}
            batch = {k: np.pad(v, ((0, pad_size),) + ((0, 0),) * (v.ndim - 1), constant_values=pad_values[k]) for k, v in batch.items()}
        return {k: torch.from_numpy(v) for k, v in batch.items()}

    def _iter_test(self):
        for set_name, dataset in self._data.items():
            total_examples = len(dataset["inputs"])
            for start_index in range(0, total_examples, self.config.global_batch_size):
                end_index = min(total_examples, start_index + self.config.global_batch_size)
                local_start, local_end = start_index, min(start_index + self.local_batch_size, end_index)
                puzzle_indices = []; puzzle_index = np.searchsorted(dataset["puzzle_indices"], local_start, side="right") - 1
                for i in range(local_start, local_end):
                    while puzzle_index + 1 < len(dataset["puzzle_indices"]) and i >= dataset["puzzle_indices"][puzzle_index + 1]: puzzle_index += 1
                    puzzle_indices.append(puzzle_index)
                yield set_name, self._collate({"inputs": dataset["inputs"][local_start:local_end], "labels": dataset["labels"][local_start:local_end],
                    "puzzle_identifiers": dataset["puzzle_identifiers"][puzzle_indices]}), end_index - start_index

    def _iter_train(self):
        for set_name, dataset in self._data.items():
            self._iters += 1
            rng = np.random.Generator(np.random.Philox(seed=self.config.seed + self._iters))
            group_order = np.concatenate([rng.permutation(dataset["group_indices"].size - 1) for _ in range(self.config.epochs_per_iter)])
            start_index = 0
            while start_index < group_order.size:
                start_index, batch_indices, batch_puzzle_indices = _sample_batch(
                    rng, group_order, dataset["puzzle_indices"], dataset["group_indices"], start_index, self.config.global_batch_size)
                if batch_puzzle_indices.size < self.config.global_batch_size: break
                yield set_name, self._collate({"inputs": dataset["inputs"][batch_indices[:self.local_batch_size]],
                    "labels": dataset["labels"][batch_indices[:self.local_batch_size]],
                    "puzzle_identifiers": dataset["puzzle_identifiers"][batch_puzzle_indices[:self.local_batch_size]]}), batch_puzzle_indices.size

    def __iter__(self):
        self._lazy_load()
        yield from (self._iter_test() if self.config.test_set_mode else self._iter_train())

print("✅ Dataset loaded")

In [ ]:
# ============================================================================
# Efficiency Metrics (NEW) - Fixed v2
# ============================================================================
import gc

def get_gpu_memory_gb():
    return torch.cuda.memory_allocated() / (1024 ** 3) if torch.cuda.is_available() else 0.0

def get_peak_memory_gb():
    return torch.cuda.max_memory_allocated() / (1024 ** 3) if torch.cuda.is_available() else 0.0

def reset_peak_memory():
    if torch.cuda.is_available(): torch.cuda.reset_peak_memory_stats()

def clear_cuda_cache():
    """Clear CUDA cache to prevent OOM between experiments"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()

def estimate_flops(batch_size, seq_len, hidden_size, num_heads, expansion, H_cycles, L_cycles, L_layers, puzzle_emb_len=16):
    """Estimate FLOPs per forward pass (single ACT step)"""
    L = seq_len + puzzle_emb_len
    D = hidden_size
    # Attention: QKV + scores + output
    attn_flops = batch_size * L * (3 * D * D + D * L + D * D)
    # MLP: SwiGLU
    inter = (-(round(expansion * D * 2 / 3) // -256)) * 256
    mlp_flops = batch_size * L * (D * inter * 2 + inter * D)
    layer_flops = attn_flops + mlp_flops
    # Total: H_cycles * (L_cycles + 1) * L_layers
    total_layer_calls = H_cycles * (L_cycles + 1) * L_layers
    return total_layer_calls * layer_flops

# Global sample counter - thread safe version
class SampleCounter:
    def __init__(self):
        self._samples = 0
    def reset(self):
        self._samples = 0
    def update(self, n):
        self._samples += n
        return self._samples
    @property
    def value(self):
        return self._samples

_sample_counter = SampleCounter()
def reset_samples(): _sample_counter.reset()
def update_samples(n): return _sample_counter.update(n)
def get_samples(): return _sample_counter.value

print("✅ Efficiency metrics loaded")

In [ ]:
# ============================================================================
# Training Framework (pretrain.py) - Faithful reproduction
# ============================================================================
class PretrainConfig(pydantic.BaseModel):
    arch: dict; data_paths: List[str]; data_paths_test: List[str] = []; evaluators: List[dict] = []
    global_batch_size: int; epochs: int; lr: float; lr_min_ratio: float; lr_warmup_steps: int
    weight_decay: float; beta1: float; beta2: float; puzzle_emb_lr: float; puzzle_emb_weight_decay: float
    project_name: Optional[str] = None; run_name: Optional[str] = None; checkpoint_path: Optional[str] = None
    seed: int = 0; checkpoint_every_eval: bool = False; eval_interval: Optional[int] = None
    min_eval_interval: int = 0; eval_save_outputs: List[str] = []; max_eval_batches: Optional[int] = None
    ema: bool = False; ema_rate: float = 0.999; freeze_weights: bool = False

@dataclass
class TrainState:
    model: nn.Module; optimizers: Sequence; optimizer_lrs: Sequence; carry: Any; step: int; total_steps: int

def create_dataloader(config, split, **kwargs):
    dataset = PuzzleDataset(PuzzleDatasetConfig(seed=config.seed, 
        dataset_paths=config.data_paths_test if config.data_paths_test and split=="test" else config.data_paths, **kwargs), split)
    return DataLoader(dataset, batch_size=None, num_workers=1, prefetch_factor=8, pin_memory=True, persistent_workers=True), dataset.metadata

def create_model(config, train_metadata):
    model_cfg = {**{k: v for k, v in config.arch.items() if k not in ['name', 'loss']},
                 'batch_size': config.global_batch_size, 'vocab_size': train_metadata.vocab_size,
                 'seq_len': train_metadata.seq_len, 'num_puzzle_identifiers': train_metadata.num_puzzle_identifiers, 'causal': False}
    with torch.device("cuda"):
        model = TinyRecursiveReasoningModel_ACTV1(model_cfg)
        print(model)
        model = ACTLossHead(model, config.arch['loss'].get('loss_type', 'stablemax_cross_entropy'))
        if "DISABLE_COMPILE" not in os.environ:
            model = torch.compile(model)
    # Optimizers (AdamW instead of AdamAtan2)
    if config.arch.get('puzzle_emb_ndim', 0) == 0:
        optimizers = [torch.optim.AdamW(model.parameters(), lr=0, weight_decay=config.weight_decay, betas=(config.beta1, config.beta2))]
        optimizer_lrs = [config.lr]
    elif config.freeze_weights:
        optimizers = [CastedSparseEmbeddingSignSGD(model.model.puzzle_emb.buffers(), lr=0, weight_decay=config.puzzle_emb_weight_decay)]
        optimizer_lrs = [config.puzzle_emb_lr]
    else:
        optimizers = [
            CastedSparseEmbeddingSignSGD(model.model.puzzle_emb.buffers(), lr=0, weight_decay=config.puzzle_emb_weight_decay),
            torch.optim.AdamW(model.parameters(), lr=0, weight_decay=config.weight_decay, betas=(config.beta1, config.beta2))]
        optimizer_lrs = [config.puzzle_emb_lr, config.lr]
    return model, optimizers, optimizer_lrs

def cosine_schedule_with_warmup(current_step, base_lr, num_warmup_steps, num_training_steps, min_ratio=0.0, num_cycles=0.5):
    """Cosine learning rate schedule with warmup - matches TinyRecursiveModels implementation"""
    if current_step < num_warmup_steps:
        return base_lr * float(current_step) / float(max(1, num_warmup_steps))
    progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
    return base_lr * (min_ratio + max(0.0, (1 - min_ratio) * 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))))

def train_batch(config, train_state, batch, global_batch_size, log_efficiency=False):
    train_state.step += 1
    if train_state.step > train_state.total_steps: return None
    
    if log_efficiency: start_time = time.time(); reset_peak_memory()
    
    batch = {k: v.cuda() for k, v in batch.items()}
    if train_state.carry is None:
        with torch.device("cuda"): train_state.carry = train_state.model.initial_carry(batch)
    
    if log_efficiency: fwd_start = time.time()
    train_state.carry, loss, metrics, _, _ = train_state.model(carry=train_state.carry, batch=batch, return_keys=[])
    if log_efficiency: fwd_time = time.time() - fwd_start
    
    samples_now = update_samples(global_batch_size)
    ((1 / global_batch_size) * loss).backward()
    
    lr_this_step = None
    for optim, base_lr in zip(train_state.optimizers, train_state.optimizer_lrs):
        lr_this_step = cosine_schedule_with_warmup(train_state.step, base_lr, config.lr_warmup_steps, train_state.total_steps, config.lr_min_ratio)
        for pg in optim.param_groups: pg['lr'] = lr_this_step
        optim.step(); optim.zero_grad()
    
    if len(metrics):
        metric_keys = sorted(metrics.keys())
        metric_values = torch.stack([metrics[k] for k in metric_keys]).cpu().numpy()
        reduced_metrics = {k: metric_values[i] for i, k in enumerate(metric_keys)}
        count = max(reduced_metrics["count"], 1)
        reduced_metrics = {f"train/{k}": v / (global_batch_size if k.endswith("loss") else count) for k, v in reduced_metrics.items()}
        reduced_metrics["train/lr"] = lr_this_step
        reduced_metrics["train/samples_seen"] = samples_now
        
        if log_efficiency:
            batch_time = time.time() - start_time
            reduced_metrics["train/batch_time"] = batch_time
            reduced_metrics["train/forward_time"] = fwd_time
            reduced_metrics["train/samples_per_sec"] = global_batch_size / batch_time
            reduced_metrics["train/gpu_memory_gb"] = get_gpu_memory_gb()
            reduced_metrics["train/peak_memory_gb"] = get_peak_memory_gb()
        return reduced_metrics
    return None

def evaluate(config, train_state, eval_loader, eval_metadata, max_batches=None):
    """Evaluate model with proper memory management - limits to max_batches for efficiency"""
    reduced_metrics = None
    
    # Print evaluation limit
    if max_batches:
        print(f"  📊 Evaluating on {max_batches} batches (limited for efficiency)")
    else:
        print(f"  📊 Evaluating on full dataset")
    
    with torch.inference_mode():
        set_ids = {k: idx for idx, k in enumerate(eval_metadata.sets)}
        metric_keys, metric_values = [], None
        processed_batches = 0
        
        for set_name, batch, global_batch_size in eval_loader:
            # Check limit before processing (ensures exactly max_batches are processed)
            if max_batches and processed_batches >= max_batches:
                break
            
            processed_batches += 1
            
            batch = {k: v.cuda() for k, v in batch.items()}
            with torch.device("cuda"): carry = train_state.model.initial_carry(batch)
            
            inference_steps = 0
            while True:
                carry, loss, metrics, preds, all_finish = train_state.model(carry=carry, batch=batch, return_keys=[])
                inference_steps += 1
                if all_finish: break
            
            if processed_batches % 10 == 0 or processed_batches == max_batches:
                print(f"  Batch {processed_batches}/{max_batches if max_batches else 'all'}: {inference_steps} ACT steps")
            
            set_id = set_ids.get(set_name, 0)
            
            if metric_values is None:
                metric_keys = sorted(metrics.keys())
                metric_values = torch.zeros((len(set_ids), len(metrics)), dtype=torch.float32, device="cuda")
            metric_values[set_id] += torch.stack([metrics[k] for k in metric_keys])
            
            # Clean up to prevent memory buildup
            del carry, loss, preds, batch, all_finish, metrics
        
        if metric_values is not None:
            reduced_metrics = metric_values.cpu().numpy()
            reduced_metrics = {set_name: {mk: reduced_metrics[si, mi] for mi, mk in enumerate(metric_keys)} for si, set_name in enumerate(set_ids)}
            for set_name, m in reduced_metrics.items():
                count = max(m.pop("count", 1), 1)  # Ensure count >= 1 to avoid division by zero
                reduced_metrics[set_name] = {k: v / count for k, v in m.items()}
        
        print(f"  ✅ Evaluation complete: {processed_batches} batches processed")
    
    return reduced_metrics

print("✅ Training framework loaded")

In [ ]:
# ============================================================================
# Dataset Builder
# ============================================================================
def shuffle_sudoku(board, solution):
    digit_map = np.pad(np.random.permutation(np.arange(1, 10)), (1, 0))
    transpose_flag = np.random.rand() < 0.5
    bands = np.random.permutation(3); row_perm = np.concatenate([b * 3 + np.random.permutation(3) for b in bands])
    stacks = np.random.permutation(3); col_perm = np.concatenate([s * 3 + np.random.permutation(3) for s in stacks])
    mapping = np.array([row_perm[i // 9] * 9 + col_perm[i % 9] for i in range(81)])
    def apply(x): return digit_map[(x.T if transpose_flag else x).flatten()[mapping].reshape(9, 9)]
    return apply(board), apply(solution)

def build_dataset(config_name, output_dir=None, force=False):
    from huggingface_hub import hf_hub_download
    cfg = DATASET_CONFIGS[config_name]
    output_dir = output_dir or f"data/sudoku-{config_name}"
    if os.path.exists(os.path.join(output_dir, "train", "dataset.json")) and not force:
        print(f"✅ Dataset exists: {output_dir}"); return output_dir
    if force and os.path.exists(output_dir): shutil.rmtree(output_dir)
    print(f"📦 Building: {config_name}")
    for sn in ["train", "test"]:
        csv_path = hf_hub_download("sapientinc/sudoku-extreme", f"{sn}.csv", repo_type="dataset", token=HF_TOKEN)
        inputs, labels = [], []
        with open(csv_path, newline="") as f:
            reader = csv.reader(f); next(reader)
            for _, q, a, _ in tqdm(reader, desc=f"Reading {sn}"):
                inputs.append(np.frombuffer(q.replace('.', '0').encode(), np.uint8).reshape(9, 9) - ord('0'))
                labels.append(np.frombuffer(a.encode(), np.uint8).reshape(9, 9) - ord('0'))
        if sn == "train" and cfg["subsample"] and cfg["subsample"] < len(inputs):
            # Set random seed for reproducibility (matches TinyRecursiveModels)
            rng_subsample = np.random.Generator(np.random.Philox(seed=42))
            idx = rng_subsample.choice(len(inputs), cfg["subsample"], replace=False)
            inputs, labels = [inputs[i] for i in idx], [labels[i] for i in idx]
        num_aug = cfg["augmentation"] if sn == "train" else 0
        results = {k: [] for k in ["inputs", "labels", "puzzle_identifiers", "puzzle_indices", "group_indices"]}
        puzzle_id, example_id = 0, 0
        results["puzzle_indices"].append(0); results["group_indices"].append(0)
        for orig_inp, orig_out in tqdm(zip(inputs, labels), total=len(inputs), desc=f"Augmenting {sn}"):
            for aug_idx in range(1 + num_aug):
                inp, out = (orig_inp, orig_out) if aug_idx == 0 else shuffle_sudoku(orig_inp, orig_out)
                results["inputs"].append(inp); results["labels"].append(out); example_id += 1; puzzle_id += 1
                results["puzzle_indices"].append(example_id); results["puzzle_identifiers"].append(0)
            results["group_indices"].append(puzzle_id)
        def to_np(seq): return np.concatenate(seq).reshape(len(seq), -1) + 1
        results = {"inputs": to_np(results["inputs"]), "labels": to_np(results["labels"]),
                   "group_indices": np.array(results["group_indices"], np.int32),
                   "puzzle_indices": np.array(results["puzzle_indices"], np.int32),
                   "puzzle_identifiers": np.array(results["puzzle_identifiers"], np.int32)}
        # Calculate mean_puzzle_examples correctly: each puzzle has (1 + num_aug) examples
        mean_puzzle_examples = 1.0 + num_aug if sn == "train" else 1.0
        meta = PuzzleDatasetMetadata(seq_len=81, vocab_size=11, pad_id=0, ignore_label_id=0, blank_identifier_id=0,
            num_puzzle_identifiers=1, total_groups=len(results["group_indices"]) - 1, mean_puzzle_examples=mean_puzzle_examples,
            total_puzzles=len(results["group_indices"]) - 1, sets=["all"])
        save_dir = os.path.join(output_dir, sn); os.makedirs(save_dir, exist_ok=True)
        with open(os.path.join(save_dir, "dataset.json"), "w") as f: json.dump(meta.model_dump(), f)
        for k, v in results.items(): np.save(os.path.join(save_dir, f"all__{k}.npy"), v)
        print(f"  ✅ {sn}: {results['inputs'].shape[0]:,} examples")
    return output_dir

print("✅ Dataset builder loaded")

In [ ]:
# ============================================================================
# Launch Function (with EMA support) - Fixed v2
# ============================================================================
import coolname

def launch(config_name, batch_size=768, project_name="TRM-DataVS", use_ema=True):
    """Launch training experiment with proper cleanup and error handling"""
    
    # Clear CUDA cache before starting
    clear_cuda_cache()
    
    cfg_d = DATASET_CONFIGS[config_name]
    data_dir = build_dataset(config_name, force=FORCE_REBUILD)
    
    config = PretrainConfig(
        arch={'halt_exploration_prob': 0.1, 'halt_max_steps': 16, 'H_cycles': 3, 'L_cycles': 6,
              'H_layers': 0, 'L_layers': 2, 'hidden_size': 512, 'num_heads': 8, 'expansion': 4,
              'puzzle_emb_ndim': 512, 'pos_encodings': 'rope', 'forward_dtype': 'bfloat16',
              'mlp_t': False, 'puzzle_emb_len': 16, 'no_ACT_continue': True,
              'loss': {'loss_type': 'stablemax_cross_entropy'}},
        data_paths=[data_dir], global_batch_size=batch_size, epochs=cfg_d['epochs'],
        eval_interval=max(cfg_d['epochs'] // 10, 1), lr=1e-4, lr_min_ratio=1.0, lr_warmup_steps=2000,
        weight_decay=1.0, beta1=0.9, beta2=0.95, puzzle_emb_lr=1e-4, puzzle_emb_weight_decay=1.0,
        project_name=project_name, run_name=f"sudoku-{config_name}",
        checkpoint_path=f"checkpoints/{project_name}/sudoku-{config_name}",
        checkpoint_every_eval=True, max_eval_batches=MAX_EVAL_BATCHES, ema=use_ema, ema_rate=0.999)
    
    torch.random.manual_seed(config.seed)
    train_epochs_per_iter = config.eval_interval or config.epochs
    train_loader, train_meta = create_dataloader(config, "train", test_set_mode=False, epochs_per_iter=train_epochs_per_iter, global_batch_size=config.global_batch_size)
    
    try: 
        eval_loader, eval_meta = create_dataloader(config, "test", test_set_mode=True, epochs_per_iter=1, global_batch_size=config.global_batch_size)
    except Exception as e: 
        print(f"⚠️ No eval data found: {e}")
        eval_loader, eval_meta = None, None
    
    model, optimizers, optimizer_lrs = create_model(config, train_meta)
    total_steps = int(config.epochs * train_meta.total_groups * train_meta.mean_puzzle_examples / config.global_batch_size)
    train_state = TrainState(model=model, optimizers=optimizers, optimizer_lrs=optimizer_lrs, carry=None, step=0, total_steps=total_steps)
    
    # EMA setup - works with compiled models
    ema_helper = None
    if config.ema:
        print('🔄 Setup EMA')
        ema_helper = EMAHelper(mu=config.ema_rate)
        ema_helper.register(train_state.model)
    
    pbar = tqdm(total=total_steps)
    wandb.init(project=config.project_name, name=config.run_name, config=config.model_dump(), settings=wandb.Settings(_disable_stats=True))
    
    # Log initial metrics
    num_params = sum(p.numel() for p in model.parameters())
    flops_estimate = estimate_flops(config.global_batch_size, train_meta.seq_len, 512, 8, 4, 3, 6, 2)
    wandb.log({"num_params": num_params, "estimated_flops_per_forward": flops_estimate}, step=0)
    
    reset_samples()
    print("="*60)
    print(f"🚀 {config.run_name}")
    print(f"   Epochs: {config.epochs}, Steps: {total_steps:,}, Params: {num_params:,}")
    print(f"   EMA: {config.ema}, FLOPs/fwd: {flops_estimate/1e9:.2f}G")
    print("="*60)
    
    try:
        total_iters = config.epochs // train_epochs_per_iter
        for iter_id in range(total_iters):
            print(f"\nEpoch {iter_id * train_epochs_per_iter}")
            train_state.model.train()
            for step_in_iter, (_, batch, gbs) in enumerate(train_loader):
                log_eff = (train_state.step % 50 == 0)  # Log efficiency every 50 steps
                metrics = train_batch(config, train_state, batch, gbs, log_efficiency=log_eff)
                if metrics:
                    wandb.log(metrics, step=train_state.step)
                    pbar.update(train_state.step - pbar.n)
                if config.ema:
                    ema_helper.update(train_state.model)
            
            # Evaluation (with EMA if enabled)
            if eval_loader and iter_id >= config.min_eval_interval:
                print("EVALUATE")
                train_state.model.eval()
                
                if config.ema:
                    print("SWITCH TO EMA")
                    # Create eval state without deepcopy (avoids issues with compiled models)
                    train_state_eval = TrainState(
                        model=ema_helper.ema_copy(train_state.model),
                        optimizers=train_state.optimizers,
                        optimizer_lrs=train_state.optimizer_lrs,
                        carry=None,  # Fresh carry for eval
                        step=train_state.step,
                        total_steps=train_state.total_steps
                    )
                else:
                    train_state_eval = train_state
                
                train_state_eval.model.eval()
                metrics = evaluate(config, train_state_eval, eval_loader, eval_meta, config.max_eval_batches)
                if metrics: wandb.log(metrics, step=train_state.step)
                
                if config.checkpoint_path and config.checkpoint_every_eval:
                    os.makedirs(config.checkpoint_path, exist_ok=True)
                    torch.save(train_state_eval.model.state_dict(), os.path.join(config.checkpoint_path, f"step_{train_state.step}"))
                
                if config.ema: 
                    del train_state_eval
                    clear_cuda_cache()  # Clear cache after eval with EMA
                    
    except KeyboardInterrupt:
        print("\n⚠️ Training interrupted by user")
    except Exception as e:
        print(f"\n❌ Training error: {e}")
        raise
    finally:
        wandb.finish()
        pbar.close()
        final_samples = get_samples()
        print(f"\n✅ Done! Samples seen: {final_samples:,}")
        clear_cuda_cache()  # Final cleanup
    
    return train_state

print("✅ Launch function ready")

In [ ]:
# ============================================================================
# 🔍 Verification - Test all components before training
# ============================================================================
def verify_setup():
    """Verify all components are working before starting experiments"""
    print("🔍 Verifying setup...")
    errors = []
    
    # 1. Check CUDA
    if not torch.cuda.is_available():
        errors.append("❌ CUDA not available")
    else:
        print(f"✅ CUDA available: {torch.cuda.get_device_name(0)}")
        print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # 2. Check imports
    required = ['numpy', 'torch', 'einops', 'pydantic', 'wandb', 'tqdm']
    for mod in required:
        try:
            __import__(mod)
            print(f"✅ {mod} imported")
        except ImportError as e:
            errors.append(f"❌ {mod} import failed: {e}")
    
    # 3. Test model instantiation (small test)
    try:
        test_cfg = {
            'batch_size': 2, 'seq_len': 81, 'vocab_size': 11,
            'num_puzzle_identifiers': 1, 'H_cycles': 1, 'L_cycles': 1,
            'H_layers': 0, 'L_layers': 1, 'hidden_size': 64, 'num_heads': 4,
            'expansion': 2, 'pos_encodings': 'rope', 'halt_max_steps': 1,
            'halt_exploration_prob': 0.0, 'puzzle_emb_ndim': 0
        }
        with torch.device("cuda"):
            test_model = TinyRecursiveReasoningModel_ACTV1(test_cfg)
        del test_model
        clear_cuda_cache()
        print("✅ Model instantiation OK")
    except Exception as e:
        errors.append(f"❌ Model instantiation failed: {e}")
    
    # 4. Test dataset config
    try:
        _ = PuzzleDatasetMetadata(
            seq_len=81, vocab_size=11, pad_id=0, ignore_label_id=0,
            blank_identifier_id=0, num_puzzle_identifiers=1,
            total_groups=100, mean_puzzle_examples=1, total_puzzles=100, sets=["all"]
        )
        print("✅ Dataset metadata OK")
    except Exception as e:
        errors.append(f"❌ Dataset metadata failed: {e}")
    
    # 5. Test efficiency metrics
    try:
        reset_samples()
        update_samples(100)
        assert get_samples() == 100
        reset_samples()
        print("✅ Sample counter OK")
    except Exception as e:
        errors.append(f"❌ Sample counter failed: {e}")
    
    # Summary
    print("\n" + "="*50)
    if errors:
        print("❌ VERIFICATION FAILED:")
        for err in errors:
            print(f"   {err}")
        return False
    else:
        print("✅ ALL CHECKS PASSED - Ready to train!")
        return True

# Run verification
verify_setup()

---
## 3️⃣ Run Experiment

In [ ]:
# ============================================================================
# 🚀 RUN EXPERIMENT
# ============================================================================
train_state = launch(
    config_name=SELECTED_CONFIG,
    batch_size=BATCH_SIZE,
    project_name=PROJECT_NAME,
    use_ema=USE_EMA
)

---
## 4️⃣ Run All Experiments (Optional)

In [ ]:
# ============================================================================
# 🚀 RUN ALL EXPERIMENTS (Uncomment to run)
# ============================================================================
# CONFIGS_TO_RUN = ["1k-aug1000", "10k-aug100", "100k-aug10", "1M-aug0"]

# results = []
# for cfg_name in CONFIGS_TO_RUN:
#     print(f"\n{'='*60}\n🚀 Starting: {cfg_name}\n{'='*60}")
#     try:
#         ts = launch(cfg_name, batch_size=BATCH_SIZE, project_name=PROJECT_NAME, use_ema=USE_EMA)
#         results.append((cfg_name, "✅ SUCCESS", ts))
#     except Exception as e:
#         print(f"❌ Failed: {cfg_name} - {e}")
#         results.append((cfg_name, f"❌ FAILED: {e}", None))
#     finally:
#         # Clean up between experiments to prevent OOM
#         clear_cuda_cache()
#         print(f"🧹 Cache cleared after {cfg_name}")

# print(f"\n{'='*60}\n🎉 ALL EXPERIMENTS COMPLETE!\n{'='*60}")
# for name, status, _ in results: 
#     print(f"  {status}: {name}")